# Import Library

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd

# Define User account

In [ ]:
search_job_name = ' '           # Bidang Industrinya
your_username = " "             # Akun LinkedIn
your_password = " "             # Password akun LinkedIn

# each page contains 25 jobs
how_many_page = 20              # Discrapping sebanyak 20 halaman

Jika ada element yang tidak ditemukan, bisa disesuaikan dengan halaman linkedin yang dimiliki karena halaman linkedin bisa berubah sewaktu-waktu.

# Scrapping Code

In [ ]:
company_name_list = []
job_location_list = []
job_title_list = []
job_desc_list = []
job_url_list = []

# Creating a webdriver instance
driver = webdriver.Chrome()

# Opening linkedIn's login page
driver.get("https://www.linkedin.com/login")

# waiting for the page to load
driver.implicitly_wait(30)

# entering username
username = driver.find_element(By.ID, "username")

# Enter Your Email Address
username.send_keys(your_username)  

# entering password
pword = driver.find_element(By.ID, "password")

# Enter Your Password
pword.send_keys(your_password)        

# Clicking on the log in button
driver.find_element(By.XPATH, "//button[@type='submit']").click()

# delay 45 seconds if there's captcha or verification, please solve before time ran out
time.sleep(45)

# Find and click job page
jobs = driver.find_element(By.XPATH, "//a[contains(@href, '/jobs')]")

jobs.click()


# search job
job_search = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located(
        (By.CSS_SELECTOR, "input[componentkey='jobSearchBox']"))
)

job_search.send_keys(search_job_name)
job_search.send_keys(Keys.RETURN)
time.sleep(1)

# Click filter button
filters_button = driver.find_element(
    By.CSS_SELECTOR,
    'button[aria-label="Show all filters. Clicking this button displays all available filter options."]'
)
filters_button.click()

# click most recent filter
most_recent = driver.find_element(By.CSS_SELECTOR, 'label[for="advanced-filter-sortBy-DD"]')
most_recent.click()

# click apply filter button
apply_filter_button = driver.find_element(
    By.CSS_SELECTOR,
    'button[aria-label="Apply current filters to show results"]'
)
apply_filter_button.click()

# loop through pages according to input
for i in range(how_many_page):
    time.sleep(5)
    button_as_reference = driver.find_element(By.CSS_SELECTOR, "button.scaffold-layout__list-jump-button")

    scrollable = button_as_reference.find_element(By.XPATH, "./ancestor::div[1]")

    scroll_height = driver.execute_script("return arguments[0].scrollHeight", scrollable)

    # scroll the job card container to trigger the lazy loading
    for offset in range(0, scroll_height, 300):
        driver.execute_script("arguments[0].scrollTop = arguments[1];", scrollable, offset)
        time.sleep(0.8)

    job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container--clickable")

    # loop to click all job card and obtain the job informations 
    for i, card in enumerate(job_cards):
        try:
            # Scroll card into view
            driver.execute_script("arguments[0].scrollIntoView(true);", card)
            time.sleep(1)
            
            # Click the job card
            card.click()
            print(f"Clicked job card #{i+1}")

            # Wait for job details panel to appear
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.ID, "job-details"))
            )

            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "//p[@dir='ltr']"))
            )
            
            
            # Extract company_name
            company_name = driver.find_element(By.CSS_SELECTOR, "div.job-details-jobs-unified-top-card__company-name").text

            # Extract job location
            job_details_container = driver.find_element(By.CLASS_NAME, "job-details-jobs-unified-top-card__tertiary-description-container")

            job_location = job_details_container.find_element(By.CSS_SELECTOR, "span.tvm__text.tvm__text--low-emphasis").text

            # Extract job title text
            job_title = driver.find_element(By.CSS_SELECTOR, "div.job-details-jobs-unified-top-card__job-title").text

            # Extract job details text
            job_desc = driver.find_element(By.ID, "job-details").text
            job_url = driver.current_url

            # Insert job informations to list
            company_name_list.append(company_name)
            job_location_list.append(job_location)
            job_title_list.append(job_title)
            job_desc_list.append(job_desc)
            job_url_list.append(job_url)

            time.sleep(2)

        # any error in finding element will result in None data row
        except Exception as e:
            print(f"⚠️ Skipping card #{i+1}: {e}")
            company_name_list.append(None)
            job_location_list.append(None)
            job_title_list.append(None)
            job_desc_list.append(None)
            job_url_list.append(None)
            continue

    # click next page button
    next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="View next page"]')
    next_button.click()

# Convert the job informations data to dataframe
df = pd.DataFrame({
    "company_name" : company_name_list,
    "job_location" : job_location_list,
    "job_title" : job_title_list,
    "job_description" : job_desc_list,
    "job_url": job_url_list
})

# save the job informations to csv
df.to_csv(f'{search_job_name.lower().replace(" ", "_")}_job.csv', index=False)
driver.close()
print(df.duplicated().sum())
print(df.isna().sum())
df

Clicked job card #1
Clicked job card #2
Clicked job card #3
Clicked job card #4
Clicked job card #5
⚠️ Skipping card #5: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=141.0.7390.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff77d3ce8e5+80021]
	GetHandleVerifier [0x0x7ff77d3ce940+80112]
	(No symbol) [0x0x7ff77d15060f]
	(No symbol) [0x0x7ff77d15816a]
	(No symbol) [0x0x7ff77d15b26f]
	(No symbol) [0x0x7ff77d15b32f]
	(No symbol) [0x0x7ff77d1a2286]
	(No symbol) [0x0x7ff77d1d122a]
	(No symbol) [0x0x7ff77d19b056]
	(No symbol) [0x0x7ff77d1d1440]
	(No symbol) [0x0x7ff77d1f968a]
	(No symbol) [0x0x7ff77d1d1003]
	(No symbol) [0x0x7ff77d1995d1]
	(No symbol) [0x0x7ff77d19a3f3]
	GetHandleVerifier [0x0x7ff77d68dc7d+2960429]
	GetHandleVerifier [0x0x7ff77d687f3a+2936554]
	GetHandleVerifier [0x0x7ff77d